In [2]:
import os
import json
import autogen

from dotenv import load_dotenv
load_dotenv()



True

In [9]:
config_list_4v = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-vision-preview"],
    },
)


config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

gpt4_llm_config = {"config_list": config_list_gpt4, "cache_seed": 42}

### Describe an Image

Requires installing the `Pillow` package.

In [11]:
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

image_agent = MultimodalConversableAgent(
    name="image-explainer",
    max_consecutive_auto_reply=10,
    llm_config={"config_list": config_list_4v, "temperature": 0.5, "max_tokens": 300}
)

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER", # Try between ALWAYS or NEVER
    max_consecutive_auto_reply=0
)

# Ask the question with an image
user_proxy.initiate_chat(image_agent, 
                         message="""What's on the image? 
<img images/RabbitCartoonHD.png>.""")

User_proxy (to image-explainer):

What's on the image? 
<image>.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
image-explainer (to User_proxy):

The image shows an adorable, stylized cartoon rabbit with large, expressive eyes and a playful smile. The rabbit has soft-looking fur, large ears, and is depicted in mid-leap against a vibrant, colorful background that suggests a serene, grassy meadow during sunrise or sunset. The sky is filled with warm colors and a few birds can be seen in the distance. There are also some flowers and plants in the foreground, adding to the peaceful and cheerful setting of the image.

--------------------------------------------------------------------------------


### Compare images

In [12]:
# Ask the question with an image
user_proxy.initiate_chat(image_agent, 
                         message="""Which image will be more appealing to a 3yo child? Why?
<img images/AIRobot.jpeg>                          
<img images/RabbitCartoonHD.png>.""")

User_proxy (to image-explainer):

Which image will be more appealing to a 3yo child? Why?
<image>                          
<image>.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
image-explainer (to User_proxy):

The second image, which features a cute, cartoon-style bunny in a colorful and whimsical setting, would likely be more appealing to a 3-year-old child. Young children are often attracted to bright colors, simple shapes, and friendly, anthropomorphic animal characters that are common in children's media. The bunny's exaggerated features, such as large eyes and a cheerful expression, are designed to be engaging and endearing to young viewers.

The first image, which depicts a highly detailed and realistic robot holding a smartphone, might not be as immediately appealing to a very young child. The complexity and mechanical nature of the robot, along with the cooler color palette, may not capture a toddler's interes

In [13]:
# Ask the question with an image
user_proxy.initiate_chat(image_agent, 
                         message="""What's the facial expression of the person on the image? WHat can you tell about the person?
<img images/KO.png>.""")

User_proxy (to image-explainer):

What's the facial expression of the person on the image? WHat can you tell about the person?
<image>.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
image-explainer (to User_proxy):

The person in the image is smiling and appears to be in a positive or happy mood. His arms are crossed, which can sometimes indicate confidence or self-assurance. He is wearing a light blue, button-up shirt, which suggests a business casual or professional dress code. The background is black, indicating that the photo might have been taken in a studio setting or edited to have a plain background, which is common for professional headshots or promotional materials.

--------------------------------------------------------------------------------
